In [1]:
import pandas as pd
import numpy as np
import os
from sktime.classification.shapelet_based import MrSEQLClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import (plot_roc_curve, classification_report, roc_auc_score, average_precision_score,
confusion_matrix, plot_confusion_matrix, plot_precision_recall_curve)

In [2]:
path = '/Users/shaow/Desktop/DS-GA1018/Project/'
X_train = pd.read_csv(os.path.join(path, "train_test_data/X_train.csv"))
X_test = pd.read_csv(os.path.join(path, "train_test_data/X_test.csv"))
y_train = pd.read_csv(os.path.join(path, "train_test_data/y_train.csv"))
y_test = pd.read_csv(os.path.join(path, "train_test_data/y_test.csv"))

X_train.drop('Unnamed: 0', axis = 1, inplace = True)
X_test.drop('Unnamed: 0', axis = 1, inplace = True)
y_train.drop('Unnamed: 0', axis = 1, inplace = True)
y_test.drop('Unnamed: 0', axis = 1, inplace = True)

In [3]:
L = 10
def CreateRainfullDataSet(X, y, L):
    input_size = X.shape[1]
    X_train_TimeSeries = np.empty([1, L, input_size])
    y_train_TimeSeries = np.empty([1, L])
    for j in range(16, 65):
        location_labels = np.where(X.iloc[: ,j] == 1)[0]
        training_index = []
        for i in location_labels[(L - 1):len(location_labels)]:
            training_index += np.arange(i- L + 1, i + 1).tolist()
            
        X_train_TimeSeries  = np.concatenate((X_train_TimeSeries, np.array(X.iloc[training_index, :].values).reshape(-1, L, input_size)), 0)
        y_train_TimeSeries = np.concatenate((y_train_TimeSeries, np.array(y.iloc[training_index].values).reshape(-1, L)), 0)
            
    return np.transpose(X_train_TimeSeries[1:, :], (0, 2, 1)), y_train_TimeSeries[1:]

X_train_TS, y_train_TS = CreateRainfullDataSet(X_train, y_train, L)
X_test_TS, y_test_TS = CreateRainfullDataSet(X_test, y_test, L)

In [4]:
y_train = np.zeros(len(y_train_TS))
y_test = np.zeros(len(y_test_TS))
for i in range(len(y_train_TS)):
    y_train[i] = int(y_train_TS[i][-1])
for i in range(len(y_test_TS)):
    y_test[i] = int(y_test_TS[i][-1])

In [6]:
ms = MrSEQLClassifier()
ms.fit(X_train_TS, y_train)
y_pred = ms.predict(X_test_TS)

C:\Users\shaow\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Call to deprecated class MrSEQLClassifier. (MrSEQLClassifier will be removed in v0.10.0. It will be replaced with an implementation based on Numba.) -- Deprecated since version 0.8.1.
  """Entry point for launching an IPython kernel.
C:\Users\shaow\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Call to deprecated class SEQLCLF. (SEQLCLF will be removed in v0.10.0) -- Deprecated since version 0.8.1.
  """Entry point for launching an IPython kernel.
C:\Users\shaow\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


In [8]:
y_pred_prob = ms.predict_proba(X_test_TS)[:,1]

In [9]:
print(classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_prob))

              precision    recall  f1-score   support

         0.0       0.90      0.74      0.81     37763
         1.0       0.42      0.70      0.53     10138

    accuracy                           0.73     47901
   macro avg       0.66      0.72      0.67     47901
weighted avg       0.80      0.73      0.75     47901

0.7980193771887105


In [10]:
accuracy_score(y_test, y_pred)

0.7326778146594016

In [11]:
ms = MrSEQLClassifier(seql_mode="fs", symrep=['sax', 'sfa'])
ms.fit(X_train_TS, y_train)
y_pred = ms.predict(X_test_TS)

C:\Users\shaow\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Call to deprecated class MrSEQLClassifier. (MrSEQLClassifier will be removed in v0.10.0. It will be replaced with an implementation based on Numba.) -- Deprecated since version 0.8.1.
  """Entry point for launching an IPython kernel.
C:\Users\shaow\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Call to deprecated class SEQLCLF. (SEQLCLF will be removed in v0.10.0) -- Deprecated since version 0.8.1.
  """Entry point for launching an IPython kernel.
C:\Users\shaow\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Call to deprecated class AdaptedSFA. (AdaptedSFA will be removed in v0.10.0) -- Deprecated since version 0.8.1.
  
C:\Users\shaow\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


In [12]:
y_pred_prob = ms.predict_proba(X_test_TS)[:,1]
print(classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_prob))

              precision    recall  f1-score   support

         0.0       0.92      0.77      0.84     37763
         1.0       0.47      0.76      0.58     10138

    accuracy                           0.77     47901
   macro avg       0.70      0.76      0.71     47901
weighted avg       0.83      0.77      0.78     47901

0.8431330372632164


In [13]:
accuracy_score(y_test, y_pred)

0.7673952527087117